In [1]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='EPMA90YFVN2QK9ZZ', output_format='pandas')

## Question 1 : -

class ScriptData:
    
    data = pd.DataFrame 
    metadata = pd.DataFrame
    
    def __init__(self, script):
        self.script = script
    
    ## a:- Fetches intraday data for given “script”
    def fetch_intraday_data(self):        
        self.data, self.metadata = ts.get_intraday(self.script)
        return self.data.head(5)

    ## b:- Converts fetched intraday data
    def convert_intraday_data(self):
        self.data.index.name = 'timestamp' 
        convert_dict = {
                'volume': int
                }

        self.data.rename(columns = {'1. open':'open',
                                    '2. high':'high',
                                    '3. low':'low',
                                    '4. close':'close',
                                    '5. volume':'volume'}, inplace = True)
        
        self.data.reset_index(level=0, inplace=True)
        
        self.data = self.data.astype(convert_dict)
    
## Question 2 :- Indicator function
    
    def indicator1(self, df: pd.DataFrame, timeperiod: int):
        new_df = df[['timestamp']].copy()
        new_df['indicator'] = df['close'].rolling(timeperiod).mean()
        return new_df
    
    
scriptData = ScriptData('GOOGL')

scriptData.fetch_intraday_data()

scriptData.convert_intraday_data()

scriptData.indicator1(scriptData.data, 5)

,timestamp,indicator
0,2023-02-10 20:00:00,NaN
1,2023-02-10 19:45:00,NaN
2,2023-02-10 19:30:00,NaN
3,2023-02-10 19:15:00,NaN
4,2023-02-10 19:00:00,94.62800
...,...,...
95,2023-02-09 12:15:00,94.03100
96,2023-02-09 12:00:00,94.02100
97,2023-02-09 11:45:00,94.07500
98,2023-02-09 11:30:00,94.26502


In [2]:
## Question 3 :- Stratergy with script name

class Statergy:
    
    def __init__(self, script):
        self.script = script
        
        
    ## helper for Question3 - C that is dataframe Signals   
    @classmethod
    def categorise(self, row):
        
        if row['indicator_data'] < row['close_data'] :
            return 'SELL'
        elif row['indicator_data'] > row['close_data'] :
            return 'BUY'
        elif row['indicator_data'] == row['close_data'] :
            return 'NO_SIGNAL'
        
    ## Uses SciptData's "fetch_intraday_data()" method  to fetch data.
    ## Uses SciptData's "convert_intraday_data" method to convert the retrieved data.
    
    ## Here Computes indicator data on ‘close’ of ‘df’ using indicator1 function.
    
    ##"indicator_data", "close_data" columns are added to retrieved data df 
    ## So "get_script_data" will give answers of question a and b both.
    
    def get_script_data(self):
        
        scriptData = ScriptData(self.script)
        scriptData.fetch_intraday_data()
        scriptData.convert_intraday_data()
            
        indicator_response = scriptData.indicator1(scriptData.data, 5)
        
        scriptData.data['indicator_data'] = indicator_response['indicator']
        scriptData.data.rename(columns = {'close':'close_data'}, inplace = True)

        return scriptData.data
    
    ## Signals dataframe
    def get_signals(self):
        
        ## Uses SciptData's method for timestamp column
        script_df = self.get_script_data() 
        df = script_df[['timestamp']].copy()
        
        ## using classmethod helper for signal (Buy, Sell , No signal)
        df['signal'] = script_df.apply(lambda row: Statergy.categorise(row), axis=1)
        return df
    
statergy = Statergy('GOOGL')
# statergy.get_script_data() 
statergy.get_signals()

,timestamp,signal
0,2023-02-10 20:00:00,None
1,2023-02-10 19:45:00,None
2,2023-02-10 19:30:00,None
3,2023-02-10 19:15:00,None
4,2023-02-10 19:00:00,BUY
...,...,...
95,2023-02-09 12:15:00,SELL
96,2023-02-09 12:00:00,SELL
97,2023-02-09 11:45:00,SELL
98,2023-02-09 11:30:00,SELL


In [3]:
## I am still learning graphs ;) 

import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

df = Statergy('GOOGL')
df = df.get_script_data()
fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close_data'])])

fig.show()
              